In [1]:
import numpy as np
import pandas as pd
import pickle
from selenium import webdriver
import time
import os
from selenium.webdriver.common.by import By
from utils import crawling_game_results, crawling_match_url, match_df, crawling_seasons, crawling_seasons_add, crawling_all_matches
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import sys

/Users/jaeminiman/anaconda3/envs/pytorch/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# set repository path of webdriver
path = './chromedriver'

In [3]:
# test webdriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.implicitly_wait(3)
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()

In [4]:
# ex) Premier leauge 2021 season 
# URL : https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228
# region number : 252
# tournament number : 2
# season number : 8228

In [5]:
def close_pop_up(driver):
    """
    close pop-up window
    
    Args :
        driver : webdriver
    
    return :
        None
    
    """
    try:
        driver.find_elements(By.CLASS_NAME, 'webpush-swal2-close')[0].click()
    except:
        pass

In [6]:
def crawling_match_url(region_number, tournaments_number, season_number, api_delay_term=5):
    """
    find the all links of matches from a certain league
    
    Args :
        region_number : region number of the league from whoscored
        tournaments_number : tournament number of the league from whoscored
        season_number : season number of the league from whoscored
        api_delay_term : break time
    
    Returns : 
        list of all the match index(link)  

    """ 
    
    # activate webdriver
    url = 'https://www.whoscored.com/Regions/'+str(region_number)+'/Tournaments/'
    url = url+str(tournaments_number)+'/Seasons/'+str(season_number)+'/Fixtures'
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)

    close_pop_up(driver)
    time.sleep(1)

    # wait get league team datas
    match_link= []
    with tqdm(total=40, file=sys.stdout) as pbar :
        for i in range(40):
            time.sleep(api_delay_term)
            elements = driver.find_elements(By.XPATH, "//*[starts-with(@id, 'scoresBtn')]")
            for element in elements:
                match_link.append(element.get_attribute('href'))

            # click
            try : 
                a = driver.find_element(By.ID, "dayChangeBtn-prev")
                a.click()
            except : 
                break

            time.sleep(2)
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
    
    driver.close()
    return list(set(match_link))

In [7]:
    
    # activate webdriver
url = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

close_pop_up(driver)
time.sleep(1)

# wait get league team datas
match_link= []
elements = driver.find_elements(By.XPATH, "//*[starts-with(@id, 'scoresBtn')]")
print(len(elements))

20


In [8]:
elements[0].get_attribute('href')

'https://1xbet.whoscored.com/matches/1485545/live/england-premier-league-2020-2021-southampton-leeds'

In [35]:
# crawling match URLs
match_url = crawling_match_url(252, 2, 8618)

page 40: 100%|██████████| 40/40 [04:43<00:00,  7.09s/it]


In [9]:
match_url = []
match_url.append(elements[0].get_attribute('href'))

In [10]:
match_url

['https://1xbet.whoscored.com/matches/1485545/live/england-premier-league-2020-2021-southampton-leeds']

In [ ]:
"col12-lg-6 col12-m-6 col12-s-6 col12-xs-12 home small-display-on"

In [33]:
def crawling_game_results(url, api_delay_term=2):
    """
    crawling results from a match
    
    
    Args :
        url : match url
        api_delay_term : break time
    
    Returns : 
        dictionary of match results

    """ 
    # activate webdriver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    # wait get league team datas
    time.sleep(api_delay_term) 

    # close pop up
    close_pop_up(driver)
    time.sleep(1)
    
    url_preview = url.replace('live','preview')
    url_show = url.replace('live','show')
    url_matchreport = url.replace('live','matchreport')
    
    driver.get(url)
    match_log = driver.find_element(By.CSS_SELECTOR, 'div.match-centre-stats').find_elements(By.CSS_SELECTOR, 'span.match-centre-stat-value')
    home_shot = match_log[2].get_attribute("textContent").split("\t")[0]
    away_shot = match_log[3].get_attribute("textContent").split("\t")[0]
    home_possession  = match_log[4].get_attribute("textContent").split("\t")[0]
    away_possession = match_log[5].get_attribute("textContent").split("\t")[0]
    home_pass_success = match_log[6].get_attribute("textContent").split("\t")[0]
    away_pass_success = match_log[7].get_attribute("textContent").split("\t")[0]
    home_dribbles = match_log[8].get_attribute("textContent").split("\t")[0]
    away_dribbles = match_log[9].get_attribute("textContent").split("\t")[0]
    home_aerials_won = match_log[10].get_attribute("textContent").split("\t")[0]
    away_aerials_won = match_log[11].get_attribute("textContent").split("\t")[0]
    home_tackles = match_log[12].get_attribute("textContent").split("\t")[0]
    away_tackles = match_log[13].get_attribute("textContent").split("\t")[0]
    home_corners = match_log[14].get_attribute("textContent").split("\t")[0]
    away_corners = match_log[15].get_attribute("textContent").split("\t")[0]
    home_dispossessed = match_log[16].get_attribute("textContent").split("\t")[0]
    away_dispossessed = match_log[17].get_attribute("textContent").split("\t")[0]
    
    driver.get(url_preview)
    time.sleep(api_delay_term)
    # get home and away
    missing_players_home = driver.find_element(By.CSS_SELECTOR, 'div.col12-lg-6.col12-m-6.col12-s-6.col12-xs-12.home.small-display-on')
    ratings = missing_players_home.find_elements(By.CSS_SELECTOR, 'td.rating')
    home_missing_player = len(ratings)
    home_missing_player_rating = []
    for rating in ratings:
        try : home_missing_player_rating.append(float(rating.get_attribute("textContent").split("\t")[0]))
        except : continue
    home_missing_player_rating = round(np.mean(np.array(home_missing_player_rating)),2)
    

    missing_players_away = driver.find_element(By.CSS_SELECTOR, 'div.col12-lg-6.col12-m-6.col12-s-6.col12-xs-12.away.small-display-off')
    ratings = missing_players_away.find_elements(By.CSS_SELECTOR, 'td.rating')
    away_missing_player = len(ratings)
    away_missing_player_rating = []
    for rating in ratings:
        try : away_missing_player_rating.append(float(rating.get_attribute("textContent").split("\t")[0]))
        except : continue
    away_missing_player_rating = round(np.mean(np.array(away_missing_player_rating)),2)
    
    home = driver.find_element(By.XPATH, '//*[@id="match-header"]/div/div[1]/span[2]/a').get_attribute("textContent").split("\t")[0]
    away = driver.find_element(By.XPATH, '//*[@id="match-header"]/div/div[1]/span[6]/a').get_attribute("textContent").split("\t")[0]
    elements = driver.find_elements(By.CSS_SELECTOR, 'div.info-block.cleared')
    score = elements[1]
    half_home_score = score.find_elements(By.CSS_SELECTOR, 'dd')[0].get_attribute("textContent").split("\t")[0].split(':')[0]
    half_away_score = score.find_elements(By.CSS_SELECTOR, 'dd')[0].get_attribute("textContent").split("\t")[0].split(':')[1]
    full_home_score = score.find_elements(By.CSS_SELECTOR, 'dd')[1].get_attribute("textContent").split("\t")[0].split(':')[0]
    full_away_score = score.find_elements(By.CSS_SELECTOR, 'dd')[1].get_attribute("textContent").split("\t")[0].split(':')[1]
    
    kick_off = elements[2].find_elements(By.CSS_SELECTOR, 'dd')[0].get_attribute("textContent").split("\t")[0]
    date = elements[2].find_elements(By.CSS_SELECTOR, 'dd')[1].get_attribute("textContent").split("\t")[0]
    
    ########### show data
    driver.get(url_show)
    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # get home and away
    matchup_home_goals = driver.find_elements(By.CSS_SELECTOR, 'td.previous-meetings-stat')[0].get_attribute("textContent").split("\t")[0]
    matchup_away_goals = driver.find_elements(By.CSS_SELECTOR, 'td.previous-meetings-stat')[3].get_attribute("textContent").split("\t")[0]
    matchup_home_wins = driver.find_elements(By.CSS_SELECTOR, 'span.title')[0].get_attribute("textContent").split("\t")[0] 
    matchup_draw = driver.find_elements(By.CSS_SELECTOR, 'span.title')[1].get_attribute("textContent").split("\t")[0] 
    matchup_away_wins = driver.find_elements(By.CSS_SELECTOR, 'span.title')[2].get_attribute("textContent").split("\t")[0] 
    
    ########### show data
    driver.get(url_matchreport)
    # wait get league team datas
    time.sleep(api_delay_term) 
    attempt = driver.find_element(By.CSS_SELECTOR, 'div.stat-group').find_elements(By.CSS_SELECTOR, 'span.stat-value')
    home_total_att = attempt[0].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_total_att = attempt[1].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_open_att = attempt[2].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_open_att = attempt[3].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_set_att = attempt[4].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_set_att = attempt[5].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_counter_att = attempt[6].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_counter_att = attempt[7].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_pk_att = attempt[8].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_pk_att = attempt[9].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_own_att = attempt[10].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0] 
    away_own_att = attempt[11].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0] 
    
    passes = driver.find_element(By.CSS_SELECTOR, '#live-chart-stats-options')
    passes.find_elements(By.CSS_SELECTOR, 'a')[1].click()
    
    time.sleep(2)
    passes = driver.find_elements(By.CSS_SELECTOR, 'div.stat-group')[2].find_elements(By.CSS_SELECTOR, 'span.stat-value')
    home_total_passes = passes[0].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_total_passes = passes[1].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_crosses_passes = passes[2].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_crosses_passes = passes[3].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_long_balls = passes[6].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_long_balls = passes[7].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    home_short_passes = passes[8].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    away_short_passes = passes[9].find_elements(By.CSS_SELECTOR, 'span')[0].get_attribute("textContent").split("\t")[0]
    
    
    driver.close()
    
    
    # close webdriver
    game_dict = {
        'home_shot' : home_shot,
        'away_shot' : away_shot,
        'home_possession' : home_possession,
        'away_possession' : away_possession, 
        'home_pass_success' : home_pass_success, 
        'away_pass_success' : away_pass_success,
        'home_dribbles' : home_dribbles,
        'away_dribbles' : away_dribbles,
        'home_aerials_won' : home_aerials_won,
        'away_aerials_won' : away_aerials_won,
        'home_tackles' : home_tackles,
        'away_tackles' : away_tackles,
        'home_corners' : home_corners,
        'away_corners' : away_corners,
        'home_dispossessed' : home_dispossessed,
        'away_dispossessed' : away_dispossessed,
        'home_missing_player' : home_missing_player,
        'away_missing_player' : away_missing_player,
        "home_missing_player_rating": home_missing_player_rating, 
        "away_missing_player_rating": away_missing_player_rating, 
        "home": home,
        "away": away,
        'half_home_score': half_home_score,
        'half_away_score': half_away_score,
        'full_home_score': full_home_score,
        'full_away_score': full_away_score,
        'kick_off': kick_off,
        'date' : date,
        'matchup_home_goals' : matchup_home_goals,
        'matchup_away_goals' : matchup_away_goals,
        'matchup_home_wins' : matchup_home_wins,
        'matchup_draw': matchup_draw,
        'matchup_away_wins': matchup_away_wins,
        'home_total_att' : home_total_att,
        'away_total_att' : away_total_att,
        'home_open_att' : home_open_att,
        'away_open_att' : away_open_att,
        'home_set_att' : home_set_att,
        'away_set_att' : away_set_att,
        'home_counter_att' : home_counter_att,
        'away_counter_att' : away_counter_att,
        'home_pk_att' : home_pk_att,
        'away_pk_att' : away_pk_att,
        'home_own_att' : home_own_att,
        'away_own_att' : away_own_att,
        'home_total_passes' : home_total_passes,
        'away_total_passes' : away_total_passes,
        'home_crosses_passes' : home_crosses_passes,
        'away_crosses_passes' : away_crosses_passes,
        'home_long_balls' : home_long_balls,
        'away_long_balls' : away_long_balls,
        'home_short_passes' : home_short_passes,
        'away_short_passes' : away_short_passes
        }
    
    return game_dict


def match_df():
    """
    empty dataframe about features of match results.
    """
    match_df = pd.DataFrame(columns=[
        'home_shot',
        'away_shot',
        'home_possession',
        'away_possession', 
        'home_pass_success', 
        'away_pass_success',
        'home_dribbles',
        'away_dribbles',
        'home_aerials_won',
        'away_aerials_won',
        'home_tackles' ,
        'away_tackles',
        'home_corners' ,
        'away_corners' ,
        'home_dispossessed',
        'away_dispossessed',
        'home_missing_player',
        'away_missing_player',
        "home_missing_player_rating", 
        "away_missing_player_rating", 
        "home",
        "away",
        'half_home_score',
        'half_away_score',
        'full_home_score',
        'full_away_score',
        'kick_off',
        'date',
        'matchup_home_goals',
        'matchup_away_goals',
        'matchup_home_wins',
        'matchup_draw',
        'matchup_away_wins',
        'home_total_att',
        'away_total_att',
        'home_open_att',
        'away_open_att',
        'home_set_att',
        'away_set_att',
        'home_counter_att',
        'away_counter_att',
        'home_pk_att',
        'away_pk_att',
        'home_own_att',
        'away_own_att',
        'home_total_passes',
        'away_total_passes',
        'home_crosses_passes',
        'away_crosses_passes',
        'home_long_balls',
        'away_long_balls',
        'home_short_passes',
        'away_short_passes'])
    return match_df

In [34]:
# crawling match results data
# ex) Liverpool vs Leicester (2021)
temp_match_url = match_url[0]
match_result = crawling_game_results(temp_match_url)

In [35]:
match_result

{'home_shot': '18',
 'away_shot': '15',
 'home_possession': '51.3',
 'away_possession': '48.7',
 'home_pass_success': '83',
 'away_pass_success': '75',
 'home_dribbles': '9',
 'away_dribbles': '7',
 'home_aerials_won': '10',
 'away_aerials_won': '16',
 'home_tackles': '23',
 'away_tackles': '35',
 'home_corners': '4',
 'away_corners': '5',
 'home_dispossessed': '21',
 'away_dispossessed': '13',
 'home_missing_player': 4,
 'away_missing_player': 5,
 'home_missing_player_rating': 6.56,
 'away_missing_player_rating': 6.42,
 'home': 'Southampton',
 'away': 'Leeds',
 'half_home_score': '0 ',
 'half_away_score': ' 0',
 'full_home_score': '0 ',
 'full_away_score': ' 2',
 'kick_off': '18:00',
 'date': 'Tue, 18-May-21',
 'matchup_home_goals': '5',
 'matchup_away_goals': '8',
 'matchup_home_wins': 'Won (50%)',
 'matchup_draw': 'Drew (0%)',
 'matchup_away_wins': 'Won (50%)',
 'home_total_att': '18',
 'away_total_att': '15',
 'home_open_att': '12',
 'away_open_att': '10',
 'home_set_att': '6',
 'a

In [9]:
# crawling all mathces data in the league.
crawling_all_matches(path,252,2,8618,'PL2122')

page 35:  88%|████████▊ | 35/40 [04:15<00:36,  7.29s/it]
PL2122_match_url.csv: done, 380 matches
match_url 1 : cawling done
match_url 2 : cawling done
match_url 3 : cawling done
match_url 4 : cawling done
match_url 5 : cawling done
match_url 6 : cawling done
match_url 7 : cawling done
match_url 8 : cawling done
match_url 9 : cawling done
match_url 10 : cawling done
match_url 11 : cawling done
match_url 11 : error
match_url 13 : cawling done
match_url 14 : cawling done
match_url 15 : cawling done
match_url 16 : cawling done
match_url 17 : cawling done
match_url 18 : cawling done
match_url 19 : cawling done
match_url 20 : cawling done
match_url 21 : cawling done
match_url 22 : cawling done
match_url 23 : cawling done
match_url 24 : cawling done
match_url 25 : cawling done
match_url 26 : cawling done
match_url 26 : error
match_url 28 : cawling done
match_url 29 : cawling done
match_url 30 : cawling done
match_url 31 : cawling done
match_url 32 : cawling done
match_url 33 : cawling done
ma

c:\Users\jaeminiman\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jaeminiman\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


match_url 315 : cawling done
match_url 316 : cawling done
match_url 317 : cawling done
match_url 318 : cawling done
match_url 319 : cawling done
match_url 320 : cawling done
match_url 321 : cawling done
match_url 322 : cawling done
match_url 323 : cawling done
match_url 324 : cawling done
match_url 325 : cawling done
match_url 326 : cawling done
match_url 327 : cawling done
match_url 328 : cawling done
match_url 329 : cawling done
match_url 330 : cawling done
match_url 331 : cawling done
match_url 332 : cawling done
match_url 333 : cawling done
match_url 334 : cawling done
match_url 335 : cawling done
match_url 336 : cawling done
match_url 337 : cawling done
match_url 338 : cawling done
match_url 339 : cawling done
match_url 340 : cawling done
match_url 341 : cawling done
match_url 342 : cawling done
match_url 343 : cawling done
match_url 344 : cawling done
match_url 345 : cawling done
match_url 346 : cawling done
match_url 347 : cawling done
match_url 348 : cawling done
match_url 349 

()

In [13]:
crawling_seasons_add(path, "PL2122", [11, 26, 46, 56, 61, 62, 74, 77, 78, 79, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 169, 195])

there are no match_number.csv file


()

In [10]:
pd.read_csv('PL2122_match.csv').head()

,Unnamed: 0,home_shot,away_shot,home_possession,away_possession,home_pass_success,away_pass_success,home_dribbles,away_dribbles,home_aerials_won,...,home_own_att,away_own_att,home_total_passes,away_total_passes,home_crosses_passes,away_crosses_passes,home_long_balls,away_long_balls,home_short_passes,away_short_passes
0,0,14,5,43.1,56.9,84,86,11,9,8,...,0,0,470,618,22,9,52,69,393,539
1,1,15,12,34.1,65.9,79,85,15,13,17,...,0,0,351,675,23,25,54,44,274,606
2,2,9,21,42.4,57.6,73,83,8,7,25,...,0,0,354,479,15,16,63,55,276,407
3,3,22,10,62.7,37.3,87,73,5,7,19,...,0,0,603,361,38,15,57,65,508,281
4,4,16,8,61.3,38.7,82,71,5,6,17,...,0,0,557,350,20,13,62,54,474,282
